In [ ]:
""" Point of this notebook is to demonstrate a method of simiulating a very simple, made up XPS spectrum using cross section"""

**Imports**

In [ ]:
import copy
import itertools as it

import sim_xps_spectra.mol_spectra.spectrum_creator as specCreator
import sim_xps_spectra.broad_functs.create_broaden_functs as createBFuncts
import sim_xps_spectra.mol_spectra.standard_objs as molSpecObjs
import sim_xps_spectra.x_sections.yeh_lindau_db as yhLindau

import sim_xps_spectra.plotters.standard_plotters as dPlotters

**Configuration variables**

In [ ]:
#What x-values we want to generate the spectrum at
X_VALS = [x/100 for x in range(100*20)]

#We use the same broadening function for ALL atomic orbitals
FWHM = 0.5
PHOTON_ENERGY = 500
EMISSION_ANGLE = None

#All variable for one fragment needs to be stored in this object
class MolFragInfo():
    def __init__(self, label ,energies, baseIntensities):
        self.label = label
        self.energies = energies
        self.baseIntensities = baseIntensities


#Define our fragment A
fragAEnergiesCl3s = [5,8]
fragAEnergiesCl3p = [1,3,8]
fragAIntensitiesCl3s = [2,5]
fragAIntensitiesCl3p = [2,7,1]

fragALabelCl3s = molSpecObjs.MolFragLabel(fragKey="Anion", eleKey="Cl", aoKey="3s")
fragALabelCl3p = molSpecObjs.MolFragLabel(fragKey="Anion", eleKey="Cl", aoKey="3p")
fragA_Cl3s = MolFragInfo(fragALabelCl3s, fragAEnergiesCl3s, fragAIntensitiesCl3s)
fragA_Cl3p = MolFragInfo(fragALabelCl3p, fragAEnergiesCl3p, fragAIntensitiesCl3p)

#Define our fragment B

#Put all fragments in a list
ALL_FRAGS = [fragA_Cl3s, fragA_Cl3p]

**Create our input object**

In [ ]:
spectraFragA = specCreator.SpectrumFragmentStandard(fragAEnergiesCl3s, fragAIntensitiesCl3s, fragALabelCl3s)
spectraFragB = specCreator.SpectrumFragmentStandard(fragAEnergiesCl3p, fragAIntensitiesCl3p, fragALabelCl3p)
specFrags = [spectraFragA, spectraFragB]

normBFunct = createBFuncts.createNormalisedGauFunctFromCentreAndFWHM(0.0,FWHM) #1st val is position

specMaker = specCreator.SpectrumCreatorStandard(spectraFrags=specFrags, normBFunct=normBFunct,
                                                xSectionDatabase=yhLindau.YehLindauXSectionDatabase(),
                                                photonEnergy=PHOTON_ENERGY, emissionAngle=EMISSION_ANGLE,
                                                xVals=X_VALS)

**Generate the spectrum**

In [ ]:
spectraOutput = specCreator.createSpectrumFromStandardCreator(specMaker)

**Plot the spectra**

In [ ]:
dataPlotter = dPlotters.DataPlotterXPS_Standard()
allData = [spectraOutput.totalSpectralContributions]
allData.extend( spectraOutput.spectralContributions )

allDataLabels = ["Total"]
allDataLabels.extend( ["-".join(x.components) for x in spectraOutput.label] )

In [ ]:
outFig = dataPlotter.createPlot(allData, dataLabels = allDataLabels, legend=True, xLim=[0,10])